# This notebook process on CLEANED and Aligned csv DF files.

# Import / Setup

In [1]:
!pip install --quiet transformers jiwer torch evaluate --use-deprecated=legacy-resolver
!pip install --quiet datasets >=2.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio
from scipy.signal import resample
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration
import evaluate

wer  = evaluate.load('wer')

# load tokenizer, processor, feature_extractor
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")
new_tokens = ['<->', '<HS>', '<SUS>', '<SUI>', '<UNK>', '<FPN>', '<SUQ>', '<CS>'] # Check if the new tokens are already in the tokenizer's vocabulary
tokenizer.add_tokens(new_tokens)

vocabs = tokenizer.get_vocab()
conflicts = [token for token in new_tokens if token in vocabs]
if conflicts:
    print(f"Conflicting tokens: {conflicts}")
else:
    print("No conflicts with the existing vocabulary.")
for token in new_tokens:
    token_id = tokenizer.encode(token, add_special_tokens=False)
    decoded_token = tokenizer.decode(token_id)
    print(f"Token: {token} - Token ID: {token_id} - Decoded Token: {decoded_token}")

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

# update preproessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")
processor.tokenizer = tokenizer  # Update the processor's tokenizer
print("Feature extractor, tokenizer, and processor initialized successfully.")

# define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)
model.config.forced_decoder_ids = None # remove conflict with transcribe


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Feature extractor, tokenizer, and processor initialized successfully.


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

#Load Dataset

In [4]:
!pip install --quiet datasets >=2.6.1
import pandas as pd
from datasets import Dataset, Audio
from google.colab import drive
import os
import json

drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/Whisper_ASR'

def check_length(dur_dict, max_dur=30, error=20):
  over_dict = {}
  over_ids = []
  mismatched = {}
  gap = []
  # if it is not overtime: not relevant to my task
  mismatched_ids = []
  ids = []
  for k, x in dur_dict.items():
    dur = x.get('duration')[1]
    length = x.get('audio_length')
    gap = dur - length
    if length > max_dur:
      over_dict[k] = x
      over_ids.append(k)
      if gap > error: # more than original
        mismatched[k] = x
        ids.append(k)
  # return ids, mismatched, over_ids, over_dict
  return over_ids, over_dict

def csv_to_ds(csv_path, filter_ids):
    ref = pd.read_csv(csv_path)

    if filter_ids:
        fil_ref = ref[~ref['file_id'].isin(filter_ids)]

    print(len(ref),'->', len(fil_ref))
    ref = fil_ref
    dataset = {
        'audio_id': ref['file_id'].tolist(),
        'audio': ref['path'].tolist(),
        'text': ref['clean_transcript'].tolist()

    }

    ds = Dataset.from_dict(dataset).cast_column("audio", Audio(sampling_rate=16000))
    return ref, ds

# load overtime_ids and filter in batch
#
file_path ='/content/drive/MyDrive/Whisper_ASR/audio_length_check_dict'
with open(file_path, 'r') as f:
    dur = json.load(f)
filter_ids, filter_dict = check_length(dur)

print('Total overtime segment:', len(filter_ids))

# load and convert to dataset + length check
csv_files = ['batch1_500.csv', 'batch2_2159.csv']

csv_path = os.path.join(drive_path, 'csv_dev', csv_files[0]) # change the index only when you need to load a different
print('valid set')

valid_dict, valid_ds = csv_to_ds(csv_path, filter_ids)
csv_path = os.path.join(drive_path, 'csv_dev', csv_files[1]) # change the index only when you need to load a different idx
print('train set')
train_dict, train_ds = csv_to_ds(csv_path, filter_ids)
# output: train_ds, valid_ds

Mounted at /content/drive
Total overtime segment: 590
valid set
500 -> 500
train set
2159 -> 2159


# Setup Training

In [5]:
class whisper_training_dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, max_len):#daatset is huggingface dataset object
        self.dataset = dataset
        self.max_len = max_len
        self.bos_token = model.config.decoder_start_token_id

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        # audio_data = down_sample_audio(item['audio']["array"], item['audio']["sampling_rate"])
        audio = item['audio']["array"]
        inputs= feature_extractor(raw_speech=audio,
                                  sampling_rate=16000,
                                  return_tensors='pt',
                                  return_attention_mask=True)
        input_feature = inputs.input_features
        feature_attention_mask = inputs.attention_mask

        # Process the transcription
        transcription = item["text"]

        # Create labels
        labels = tokenizer(transcription, padding="max_length", max_length=self.max_len, truncation=True, return_tensors="pt")
        labels = labels["input_ids"].masked_fill(labels['attention_mask'].ne(1), -100)
        labels = labels[0][1:]

        return {
            "input_features": input_feature,
            "feature_attention_mask": feature_attention_mask,
            "labels": labels
        }

In [6]:
def evaluation(model, ds):

    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    predictions=[]
    references=[]

    for sample in tqdm(ds, total=len(ds)):
        audio=sample['audio']['array']
        sample_rate=sample['audio']['sampling_rate']
        text=sample['text']

        inputs= feature_extractor(raw_speech=audio,
                                          sampling_rate=16000,
                                          return_tensors='pt',
                                  return_attention_mask=True)
        input_feature = inputs.input_features
        feature_attention_mask = inputs.attention_mask

        with torch.no_grad():
            op = model.generate(
              input_feature.to(device),
              attention_mask=feature_attention_mask.to(device),
              language='english',
              task='transcribe'
            )

        text_pred =  tokenizer.batch_decode(op,skip_special_tokens=True )[0]
        predictions.append(text_pred)
        references.append(text)

    WER = wer.compute(predictions=predictions, references=references) * 100

    return WER, predictions, references

In [7]:
# adjust valid_ds if changing dataset

dataset = whisper_training_dataset(dataset=train_ds, max_len=448)

train_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=4,  # Adjust batch size as needed
    shuffle=True,  # Shuffle data during training
    drop_last=False
)

In [8]:
from IPython.display import clear_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyperparameters
optimizer=torch.optim.AdamW(model.parameters(), lr=1e-5)
# further implementation
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

max_epochs=3
running_wer=[]
loss = []
for epoch in tqdm(range(max_epochs)):
    print(epoch)
    l = []
    for batch in train_dataloader:

        model.train()  # Set model to training mode

        # pass info to model
        input_features = batch["input_features"].to(device).squeeze(1)  # Shape: [4, 80, 3000]
        feature_attention = batch["feature_attention_mask"].to(device).squeeze(1)  # Shape: [4, 3000]
        labels = batch["labels"].to(device)  # Shape: [4, 447]

        # Forward pass
        # error: dimension mismatch in CovV1D: process input features
        # Cov1D: batch_size, feature_dim, sequence_length
        outputs = model(input_features, feature_attention, labels=labels)  # Assuming your model takes these inputs
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        # avoid exploding gradients
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()  # Reset gradients
        print(loss.item())
        l.append(loss.item())

    wer, pred, ref = evaluation(model, valid_ds)
    running_wer.append(wer)
    loss.append(l)

    plt.plot(running_wer)
    clear_output(wait=True)
    plt.xlabel('epochs')
    plt.ylabel('wer (%)')
    plt.show()


  0%|          | 0/3 [00:00<?, ?it/s]

0


  0%|          | 0/3 [00:15<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
running_wer

[22.374877330716387, 19.88877984952568, 19.7252208047105]

In [ ]:
torch.save(model.state_dict(), 'whispter_ft_train.pth')

## Run Eval

In [ ]:
from google.colab import drive
import os
import json

drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/Whisper_ASR'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.load_state_dict(torch.load('whispter_ft_train.pth', weights_only=True))# loading the model
model.eval()

# change eval dataset
ds = valid_ds
model.to('cuda')

FileNotFoundError: [Errno 2] No such file or directory: 'whispter_ft_train.pth'

In [ ]:
predictions=[]
references=[]

# 10 min for one
for sample in tqdm(ds, total=len(ds)):
    text = sample['text']
    audio_original = sample['audio']['array']

    inputs= feature_extractor(raw_speech=audio_original,
                                      sampling_rate=16000,
                                      return_tensors='pt',
                              return_attention_mask=True)
    input_feature = inputs.input_features
    feature_attention_mask = inputs.attention_mask

    with torch.no_grad():
        op = model.generate(
          input_feature.to(device),
          attention_mask=feature_attention_mask.to(device),
          language='english',
          task='transcribe'
        )

    text_pred =  tokenizer.batch_decode(op,skip_special_tokens=True )[0]
    predictions.append(text_pred)
    references.append(text)

  1%|          | 1/83 [00:00<00:49,  1.66it/s]



true : well to be honest i am not that i'm not a big fan of <HS> environment <SUS> <HS> but however i am still care about <HS> the <HS> the environment and trees and animals <SUS> 
pred :  Well to be honest i am not that i'm not a big fan of environment but however i'm still care about the the the environment and trees and animals




  2%|▏         | 2/83 [00:01<00:50,  1.59it/s]



true : are plenty of things we really enjoy doing <HS> during our free time <SUI> so let's say for example last week we have played football game with my friends <SUS> <HS> the other one we played online ga<-> <SUI> 
pred :  There are plenty of things we really enjoy doing during our free time. So let's say for example last week we have played football game with my friends the other one we played online game




  4%|▎         | 3/83 [00:01<00:46,  1.73it/s]

...



true : in my opinion attending lectures is better than reading alone because attending lectures you have you have interactive <HS> speaking with a person or with a <UNK> <SUS> and this is better than reading alone that is bored <SUS> 
pred :  In my opinion attending lectures is better than reading alone because attending a lecture you have you have interactive speaking with a person or with a videoing and this is better than reading alone that is boring




 99%|█████████▉| 82/83 [00:49<00:00,  1.51it/s]



true : from my point of view is the most important to have learning facilities because at university if you not have a computer or another technology you cannot learn more than other students <SUS> and this is a difficult in others countries <SUS> and this is essenti<-> <SUI> 
pred :  From a point of view it's the most important to have learning facilities because at a university if you not have a computer or another technology you cannot learn more than other students and this is a difficult in other countries and this is essential




100%|██████████| 83/83 [00:49<00:00,  1.67it/s]



true : i am very interested in know about other countries <SUS> but i'm in love with barcelona and i'm in love with catalonia and spain too <SUS> 
pred :  i'm very interested in know about other countries but i'm in love with basalona and i'm in love with catalonia and spain too




# Run Baseline Pipeline

In [9]:
!pip install tqdm datasets transformers
import pandas as pd
from datasets import Dataset, Audio
from google.colab import drive
from tqdm import tqdm
import os
import json
import torch
from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperProcessor, WhisperForConditionalGeneration

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
def mount_drive():
    drive.mount('/content/drive')

def check_length(dur_dict, max_dur=30, error=20):
    over_ids = [k for k, x in dur_dict.items() if x.get('audio_length') > max_dur and (x.get('duration')[1] - x.get('audio_length')) > error]
    return over_ids

def csv_to_ds(csv_path, filter_ids):
    ref = pd.read_csv(csv_path)
    if filter_ids:
        ref = ref[~ref['file_id'].isin(filter_ids)]
    print(f"Filtered {len(ref)} records out of {len(filter_ids)}")
    dataset = {
        'audio_id': ref['file_id'].tolist(),
        'audio': ref['path'].tolist(),
        'transcript': ref['clean_transcript'].tolist()
    }
    ds = Dataset.from_dict(dataset).cast_column("audio", Audio(sampling_rate=16000))
    return ref, ds

def initialize_tokenizer_processor():
    feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small",task="transcribe")
    tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")
    new_tokens = ['<->', '<HS>', '<SUS>', '<SUI>', '<UNK>', '<FPN>', '<SUQ>', '<CS>']
    tokenizer.add_tokens(new_tokens)
    processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")
    processor.tokenizer = tokenizer
    return tokenizer, processor

def initialize_model(tokenizer):
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)
    model.resize_token_embeddings(len(tokenizer), mean_resizing=False)
    model.config.forced_decoder_ids = None
    return model

def perform_inference(ds, model, tokenizer, feature_extractor):
    predictions = []
    references = []
    for sample in tqdm(ds, total=len(ds)):
        text = sample['transcript']
        audio_original = sample['audio']['array']
        inputs = feature_extractor(raw_speech=audio_original, sampling_rate=16000, return_tensors='pt', return_attention_mask=True)
        input_feature = inputs.input_features
        feature_attention_mask = inputs.attention_mask
        with torch.no_grad():
            op = model.generate(input_feature.to(device), attention_mask=feature_attention_mask.to(device), language='english', task='transcribe')
        text_pred = tokenizer.batch_decode(op, skip_special_tokens=True)[0]
        predictions.append(text_pred)
        references.append(text)
    return predictions, references

def run_baseline_performance(drive_path, csv_files, audio_length_check_dict_path, max_dur=30, error=20):
    # Mount Drive
    mount_drive()

    # Load and filter overtime_ids
    with open(audio_length_check_dict_path, 'r') as f:
        dur = json.load(f)
    filter_ids = check_length(dur, max_dur, error)
    print('Total overtime segment:', len(filter_ids))

    # Load and filter CSV files
    csv_path = os.path.join(drive_path, 'csv_dev', csv_files[0])
    print('Valid set')
    valid_dict, valid_ds = csv_to_ds(csv_path, filter_ids)

    # Initialize tokenizer, processor, and model
    tokenizer, processor = initialize_tokenizer_processor()
    feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = initialize_model(tokenizer)

    # Perform inference
    predictions, references = perform_inference(valid_ds, model, tokenizer, feature_extractor)

    return predictions, references

In [ ]:
# example usages:
drive_path = '/content/drive/MyDrive/Whisper_ASR'

# Batch_size, dict.keys = 'file_id', 'path', 'clean_transcript'
# csv_files = ['batch1_100.csv', 'batch2_1000.csv', 'batch3_1000.csv', 'batch4_1149.csv']
csv_files = ['batch1_500.csv', 'batch2_2159.csv']

filter_audio_length_path = '/content/drive/MyDrive/Whisper_ASR/audio_length_check_dict'

predictions, references = run_baseline_performance(drive_path, csv_files, filter_audio_length_path)

# Optional: Print or save the results
# for pred, ref in zip(predictions, references):
#     print(f"Prediction: {pred}\nReference: {ref}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total overtime segment: 24
Valid set
Filtered 500 records out of 24


 12%|█▏        | 61/500 [02:09<13:39,  1.87s/it]

# Percentages of correct tags

In [ ]:
from transformers import PreTrainedTokenizerFast

# Define your special tokens
special_tokens = ['<->', '<HS>', '<SUS>', '<SUI>', '<UNK>', '<FPN>', '<SUQ>', '<CS>']

# Initialize the tokenizer (using a pre-trained tokenizer as a base)
tokenizer = PreTrainedTokenizerFast.from_pretrained('bert-base-uncased')

# Add the special tokens to the tokenizer
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

def tokenize_s(sentence):
    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)

    # Encode the sentence (convert tokens to IDs)
    encoded = tokenizer.encode(sentence)

    # Decode the IDs back to the sentence
    decoded = tokenizer.decode(encoded)

    return tokens

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
def tag_counts(ls):
    tokens = ['<->', '<HS>', '<SUS>', '<SUI>', '<UNK>', '<FPN>', '<SUQ>', '<CS>']

    # Initialize the dictionary with tokens as keys and counts as values
    counts = {token: 0 for token in tokens}

    # Tokenize each string in the list
    tokenized = [tokenize_s(s) for s in ls]

    # Update the counts based on the tokens found
    for tokens_list in tokenized:
        for token in tokens_list:
            if token in counts:
                counts[token] += 1

    return counts

In [ ]:
pred_tags = tag_counts(pred_ls)
ref_tags = tag_counts(ref_ls)
print(pred_tags)
print(ref_tags)

{'<->': 0, '<HS>': 1, '<SUS>': 1, '<SUI>': 1, '<UNK>': 1, '<FPN>': 1, '<SUQ>': 2, '<CS>': 1}
